In [2]:
# Import required libraries for environment setup and API key handling
import getpass
import os
import dotenv
from langchain_core.messages import HumanMessage
from langchain.chat_models import init_chat_model

# Load environment variables from .env file (if it exists)
dotenv.load_dotenv(override=True)

# Check if Google API key is already set in environment variables
# If not, prompt user to enter it securely (input will be hidden)
if not os.environ.get("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")


# Import LangChain's chat model initialization function

# Initialize the Gemini 2.5 Flash model
# This creates a chat model instance that we can use to interact with the AI
google_model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")
openai_model = init_chat_model("gpt-4o-mini", model_provider="openai")
anthropic_model = init_chat_model("claude-3-5-sonnet-20241022", model_provider="anthropic")
model = openai_model

print("✅ Model initialized successfully!")


✅ Model initialized successfully!


In [18]:
# Ask a follow-up question without providing conversation context
# Since the model is stateless, it won't remember that we said our name is Bob
response = model.invoke([HumanMessage(content="What's my name?")])

print("AI Response (without context):")
print(response.content)
print("\n" + "="*50)
print("Notice: The model doesn't remember the previous conversation!")
print("This makes for a terrible chatbot experience.")


AI Response (without context):
I don't have access to personal information about users unless it's provided in the conversation. So I don't know your name. If you'd like to share it, feel free!

Notice: The model doesn't remember the previous conversation!
This makes for a terrible chatbot experience.


In [16]:

# Import AIMessage to represent responses from the AI model
from langchain_core.messages import AIMessage

# Create a conversation history with multiple turns
# This simulates a real conversation where context is maintained
conversation_history = [
    HumanMessage(content="Hi! I'm Bob"),
    AIMessage(content="Hello Bob! How can I assist you today?"),
    HumanMessage(content="What's my name?"),
]

# Pass the entire conversation history to the model
# Now the model has context and can answer the follow-up question correctly
response = model.invoke(conversation_history)

print("AI Response (with conversation history):")
print(response.content)
print("\n" + "="*50)
print("Success! Now the model remembers the conversation context.")


AI Response (with conversation history):
Your name is Bob! How can I help you today, Bob?

Success! Now the model remembers the conversation context.


In [ ]:
def chat():
    conversation = []
    print("🤖 Chat (type 'quit' to exit)")
    
    while True:
        user_input = input("�� You: ")
        print(f"User input: {user_input}")
        if user_input.lower() == 'quit':
            break
        
        conversation.append(HumanMessage(content=user_input))
        response = model.invoke(conversation)
        conversation.append(response)
        print(f"🤖 AI: {response.content}")

chat()

🤖 Chat (type 'quit' to exit)
Hey how are you
🤖 AI: I'm just a program, but I'm here and ready to help you! How can I assist you today?

🤖 AI: It looks like your message didn't come through. How can I assist you?
quit


In [3]:
from typing import List, Optional
from pydantic import BaseModel, Field

# Pydantic model for a single joke
class Joke(BaseModel):
    """Joke to tell user."""
    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: Optional[int] = Field(
        default=None, description="How funny the joke is, from 1 to 10"
    )

# New Pydantic model for a list of jokes
class JokeList(BaseModel):
    """A list of jokes."""
    jokes: List[Joke] = Field(description="A list of jokes to tell the user.")

# Instantiate the structured LLM with the new model
structured_llm_multi = model.with_structured_output(JokeList)

# Invoke the LLM, specifying how many jokes you want
response = structured_llm_multi.invoke(f"""parse this text to cat's jokes {input('input here')}""")

# Now you can iterate over the list of joke objects
for joke_obj in response.jokes:
    print(f"Setup: {joke_obj.setup}")
    print(f"Punchline: {joke_obj.punchline}")
    print("-" * 20)

Setup: Why are cats so good at video games?
Punchline: Because they have 9 lives, of course!
--------------------


In [4]:
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage
from pydantic import BaseModel, Field
from typing import Optional

# 1. Define your Pydantic model
class ChatResponse(BaseModel):
    """A structured response from the chat assistant."""
    message: str = Field(description="The chat assistant's primary message.")
    sentiment: Optional[str] = Field(description="The sentiment of the user's message (e.g., positive, negative, neutral).")

# 2. Define the chat function that will use the structured model
def chat_with_pydentic_model(model):
    conversation = []
    print("🤖 Chat (type 'quit' to exit)")

    while True:
        user_input = input("🙂 You: ")
        print(f"User input: {user_input}")
        if user_input.lower() == 'quit':
            break

        # Step 1: Append the user's message to the conversation
        conversation.append(HumanMessage(content=user_input))
        
        # Step 2: Invoke the model with the updated conversation history
        response_obj = model.invoke(conversation)

        # Step 3: Append the AI's response as an AIMessage object
        conversation.append(AIMessage(content=response_obj.message))
        
        # Now, print the response from the Pydantic object
        print(f"🤖 AI: {response_obj.message}")
        if hasattr(response_obj, 'sentiment'):
            print(f"Sentiment: {response_obj.sentiment}")


# 4. Bind the Pydantic schema to the base LLM to create the structured model
structured_llm = model.with_structured_output(ChatResponse)

# 5. Call the function with the new, structured model
chat_with_pydentic_model(model=structured_llm)

🤖 Chat (type 'quit' to exit)
User input: hello how are you
🤖 AI: Hello! I'm just a program, so I don't have feelings, but I'm here to help you. How can I assist you today?
Sentiment: neutral
User input: I reely feel great today
🤖 AI: That's wonderful to hear! I'm glad you're feeling great today. Is there anything special you're doing to celebrate your good mood?
Sentiment: positive
User input: now i feel not so well
🤖 AI: I'm sorry to hear that you're not feeling well now. If you'd like to talk about it or if there's anything I can do to help, please let me know.
Sentiment: negative
User input: quit
